In [1]:
import pandas as pd
import numpy as np
import math

### Load JSON into ORtools Input Format

In [2]:
from Json_to_OR_tools import json2order, orders2Ortools

filename = "waypoints_order.json"
all_orders = json2order(filename)

subset_num = 120
all_orders = all_orders[:subset_num]

# depot_lat = 1.35, depot_lon = 103.7, depot_tw_str = "00:00-23:59", depot_demand = 0 is default specified within orders2Ortools
orders_time_matrix, pick_drop_list, time_window_list, demand_list = orders2Ortools(all_orders)

### Helper Functions
- create_data_model
- print_solution

In [3]:
'''need to parse json to extract and convert time format to time window'''

def create_data_model(pd,time_mat,tw,num_veh,demand):
    """Stores the data for the problem."""
    data = {}
    data['pickups_deliveries'] = pd
    data['time_matrix'] = time_mat
    data['time_windows'] = tw
    data['num_vehicles'] = num_veh
    data['depot'] = 0
    data['demands'] = demand
    data['vehicle_capacities'] = [2000 for i in range(data['num_vehicles'])]
    return data


In [4]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    time_dimension = routing.GetDimensionOrDie('Time')
    total_time = 0
    total_load = 0
    total_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        tardiness=0
        while not routing.IsEnd(index):
            #capacity
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
            
        
            
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                             route_load)
#         plan_output += 'Distance of the route: {}m\n'.format(route_distance)
#         plan_output += 'Load of the route: {}\n'.format(route_load)
        
        index = routing.Start(vehicle_id)
        while not routing.IsEnd(index):
            #time window
            time_var = time_dimension.CumulVar(index)
            plan_output += '{0} Time({1},{2}) -> '.format(
                manager.IndexToNode(index), solution.Min(time_var),
                solution.Max(time_var))
            if index<len(data['time_windows']) and solution.Max(time_var)> data['time_windows'][index][1]:
                tardiness+= solution.Max(time_var)-data['time_windows'][index][1]
#                 tardiness+= time_dimension.SetCumulVarSoftUpperBound(index, data['time_windows'][index][1],1) 
            index = solution.Value(routing.NextVar(index))
            
        
        time_var = time_dimension.CumulVar(index)
        plan_output += '{0} Time({1},{2})\n'.format(manager.IndexToNode(index),
                                                    solution.Min(time_var),
                                                    solution.Max(time_var))
        plan_output += 'Time of the route: {}min\n'.format(
            solution.Min(time_var))
        
        plan_output += 'Tardiness of the route: {}min\n'.format(
            tardiness)
        
        print(plan_output)
        
        total_load += route_load
        total_distance += route_distance
        total_time += solution.Min(time_var)
    print('Total time of all routes: {}min'.format(total_time))
#     print('Total distance of all routes: {}m'.format(total_distance))
#     print('Total load of all routes: {}'.format(total_load))

### Main Function

In [5]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def main(pd,time_mat,tw,num_veh,demand):
    # Instantiate the data problem.
    data = create_data_model(pd,time_mat,tw,num_veh,demand)
    
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']),
                                           data['num_vehicles'], data['depot'])
    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)
    
    # Define cost of each arc.
    def time_callback(from_index, to_index):
        """Returns the time distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]
    
    transit_callback_index = routing.RegisterTransitCallback(time_callback)
#     routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    #Add time window constraint
    dimension_name = 'Time'
    routing.AddDimension(
        transit_callback_index,
        1439,  # allow waiting time
        1439,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        dimension_name)
    time_dimension = routing.GetDimensionOrDie(dimension_name)
    
    
    # Add time window constraints for each location except depot.
    for location_idx, time_window in enumerate(data['time_windows']):
        if location_idx == data['depot']:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], 1439)
        time_dimension.SetCumulVarSoftUpperBound(index, time_window[1],1)    
        
    # Add time window constraints for each vehicle start node.
    depot_idx = data['depot']
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(
            data['time_windows'][depot_idx][0],
            data['time_windows'][depot_idx][1])
    
    for i in range(data['num_vehicles']):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i)))
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.End(i)))
    
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')
    
    # Define Transportation Requests.
    for request in data['pickups_deliveries']:
        pickup_index = manager.NodeToIndex(request[0])
        delivery_index = manager.NodeToIndex(request[1])
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        routing.solver().Add(
            routing.VehicleVar(pickup_index) == routing.VehicleVar(
                delivery_index))
        routing.solver().Add(
            time_dimension.CumulVar(pickup_index) <=
            time_dimension.CumulVar(delivery_index))
    
    routing.SetFixedCostOfAllVehicles(60)
     # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
#     search_parameters.solution_limit = 1
    search_parameters.log_search = True
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    # search_parameters.time_limit.seconds = 120

#     search_parameters.first_solution_strategy = (
#         routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)
    
    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    
    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print('Sol not found')
    


### Run Solver

In [6]:
import time
time.ctime()

'Sat Apr  2 23:02:11 2022'

In [8]:
num_veh=10

print(time_window_list)
if __name__ == '__main__':
    main(pick_drop_list,orders_time_matrix,time_window_list,num_veh,demand_list)

[(0, 1439), (840, 960), (840, 960), (480, 600), (480, 600), (540, 660), (540, 660), (900, 1020), (900, 1020), (900, 1020), (900, 1020), (900, 1020), (900, 1020), (900, 1020), (900, 1020), (900, 1020), (900, 1020), (900, 1020), (900, 1020), (840, 960), (840, 960), (840, 960), (840, 960), (840, 960), (840, 960), (900, 1020), (900, 1020), (900, 1020), (900, 1020), (900, 1020), (900, 1020), (1020, 1140), (1020, 1140), (1140, 1260), (1140, 1260), (540, 660), (540, 660), (540, 660), (540, 660), (540, 660), (540, 660), (600, 720), (600, 720), (600, 720), (600, 720), (600, 720), (600, 720), (600, 720), (600, 720), (840, 960), (840, 960), (900, 1020), (900, 1020), (900, 1020), (900, 1020), (720, 840), (720, 840), (720, 840), (720, 840), (720, 840), (720, 840), (720, 840), (720, 840), (720, 840), (720, 840), (720, 840), (720, 840), (720, 840), (720, 840), (720, 840), (720, 840), (720, 840), (720, 840), (720, 840), (720, 840), (720, 840), (720, 840), (720, 840), (720, 840), (720, 840), (720, 840)

In [ ]:
import time
time.ctime()

'Sat Apr  2 22:36:15 2022'

In [ ]:
# started at 11.21am